# Algorithms with Matrices

In this practice we are going to implement algorithms to work with matrices:
- solving simple equations
- inverting matrices
- decomposing matrices to LU decompositions

For these activities Gauss Elimination is our main tool. We will only work with matrices that do not require swapping the rows.

We will also perform QR decomposition using Gramm-Schmidt orthonormalization method

In [78]:
import numpy as np
import lovely_numpy as ln
import json_tricks 
import matplotlib.pyplot as plt

In [79]:
np.random.seed(0)


inputs = json_tricks.load('inputs/inputs.json')
answer = {}

In [80]:
print(len(inputs['nullspace_cases']))

30


# Task 1. Implement Gauss Elimination Stage I

## 1.1 Gauss Elimination -- non-pivoted case
Firstly, your task here is to create an algorithm that will perfrom mutations of matrix $A$ that transform it to Upper-Triangular form $U$.

$$
\begin{bmatrix}
a_{11} & a_{12} & \dots & a_{1N} \\
a_{21} & a_{22} & \dots & a_{2N} \\
\vdots & \vdots & \ddots & \vdots \\
a_{N1} & a_{N2} & \dots & a_{NN}
\end{bmatrix}

\rightarrow

\begin{bmatrix}
1 & a_{12}^* & \dots & a_{1N}^* \\
0 & 1 & \dots & a_{2N}^* \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \dots & 1
\end{bmatrix}
$$

Note that there should be unit diagonal after the 1 stage.

The same mutations for rows as you do to the matrix $A$ should be done to the rows of matrix $B$. 

What can matrix $B$ be? It can be:
- vector of right hand side $b$ (in that case we are solving the System of Linear Equations)
- identity matrix (in that case we invert matrix)
- actually, Stage I can be used as Stage II, and in that case right hand matrix can be some matrix from Stage I

## 1.2 Add pivoting

There is a flag `pivoting`. In case this flag is turned on, the Gauss Elimination should swap the current row with 
the row with maximal absolute value in the current column.

You should also perform the same swap with the matrix $B$ and also you should keep track of swaps with the matrix
$P$.

## 1.3 Account for the cases with null-space

In that case once in a while you will not be able to perform pivoting to evade the division-by-zero. In that case you
will find out that the diagonal element and all the elements below are all together less than some threshold value.

In that case just continue to the next column.

Keep in mind though that you have to advance only the column index, but the row index should not be increased. That
means that now your pivot is off the main diagonal.


In [81]:
def gauss_elimination(A, B, pivoting=False):
    A = A.copy().astype(float)
    B = B.copy().astype(float)

    n_rows, n_cols = A.shape
    P = np.eye(n_rows)  

    # solve SLEs
    if B.shape[0] == n_rows:
        A = np.hstack((A, B.reshape(-1, 1)))

    # find inverse
    if B.shape == (n_rows, n_cols):
        A = np.hstack((A, B))

    if pivoting:
        for i in range(n_rows):
            max_row = np.argmax(np.abs(A[i:, i])) + i
            if max_row != i:
                A[[i, max_row], :] = A[[max_row, i], :]
                P[[i, max_row], :] = P[[max_row, i], :]
    else:
        for i in range(n_rows):
            max_row = i
            for j in range(i + 1, n_rows):
                if abs(A[j, i]) > abs(A[max_row, i]):
                    max_row = j
            if max_row != i:
                A[[i, max_row], :] = A[[max_row, i], :]
                P[[i, max_row], :] = P[[max_row, i], :]
            

    return P, A, B


In [82]:
answer['gauss_elimination_simple_cases'] = [
    gauss_elimination(A=input['A'], B=input['b'], pivoting=False) for input in inputs['simple_cases']
]

answer['gauss_elimination_pivoting_cases'] = [
    gauss_elimination(A=input['A'], B=input['b'], pivoting=True) for input in inputs['simple_cases']
]

answer['gauss_elimination_nullspace_cases'] = [
    gauss_elimination(A=input['A'], B=input['b'], pivoting=True) for input in inputs['nullspace_cases']
]

# Task 2: Solve System of Linear Equations (SLE)

Now we have the engine to solve a lot of things. We will start with solving Systems of Linear Equations (SLEs).

For the SLE given by matrix $A$ and right-hand vector $\mathbf b$, implement a function that solves this SLE.

You will need to use twice Gauss Elimination Algorithm

1. Apply it to matrix $A$ and to vector $\mathbf b$ as matrix $B$ as Gauss Elimintaion I stage
2. For Gauss Elimination II stage you can use the same algorithm by just applying it to reversed matrix 

    (by enumerating equations and variables in reversed order, we can rewrite the matrix $U$ in low-triangle form $L$). 

    Something has to be done to the right-hand side vector too
3. Form the answer from output of Gauss Elimination Stage II

In [83]:
def solve_equation(A, b):
    P, A, b = gauss_elimination(A, b, pivoting=True)
    n = A.shape[0]
    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = b[i]
        for j in range(i + 1, n):
            x[i] -= A[i, j] * x[j]
        x[i] /= A[i, i]

    return x

You can always check that your solution is correct by subtracting

$A \mathbf x - \mathbf b$, which should give you $\mathbf 0$

By the way, matrix product, such as $A \mathbf x$ in numpy is done using `A @ x` operation

In [84]:
answer['solutions_simple_cases'] = [
    solve_equation(**input) for input in inputs['simple_cases']
]

answer['solution_pivoting_cases'] = [
    solve_equation(**input) for input in inputs['pivoting_cases']
]

# This should be close to zero
print("=== SIMPLE CASES ===")
for index in range(len(inputs['simple_cases'])):
    print(np.abs(inputs['simple_cases'][index]['A'] @ answer['solutions_simple_cases'][index] - inputs['simple_cases'][index]['b']).sum())

print("=== PIVOTING CASES ===")
for index in range(len(inputs['pivoting_cases'])):
    print(np.abs(inputs['pivoting_cases'][index]['A'] @ answer['solution_pivoting_cases'][index] - inputs['pivoting_cases'][index]['b']).sum())

=== SIMPLE CASES ===
3359.0
498.83100198412694
118.9
2827.2102040816317
36.5
28.0
nan
12.75
539.7511574074074
4065.4636363636364
6.0
147.72592592592594
nan
432.425925925926
nan
10.0
0.0
395.3333333333333
4.0
336.5
nan
6.0
1.3333333333333321
846.3604910714284
40.33333333333333
nan
5.0
646.8636363636363
3671.7853535353543
4.666666666666666
=== PIVOTING CASES ===
nan
1010.8848039215691
1249.9
nan
0.0
807.3511904761904
1722.4582739791076
269.58333333333337
16176.0
nan
291.1851851851852
144.0
887.296875
13129.309941520467
260.5683760683761
4219.503042117078
4440.487989472649
0.0
nan
2455.4989976631914
2743.9259999999995
27720.497835888258
17539.64888888889
nan
399.4
193.52380952380955
nan
788.4444444444445
1770.605048148148
nan


/tmp/ipykernel_11082/2294918992.py:9: RuntimeWarning: divide by zero encountered in scalar divide
  x[i] /= A[i, i]
/tmp/ipykernel_11082/2294918992.py:8: RuntimeWarning: invalid value encountered in scalar multiply
  x[i] -= A[i, j] * x[j]
/tmp/ipykernel_11082/2294918992.py:8: RuntimeWarning: invalid value encountered in scalar subtract
  x[i] -= A[i, j] * x[j]
/tmp/ipykernel_11082/728041831.py:12: RuntimeWarning: invalid value encountered in matmul
  print(np.abs(inputs['simple_cases'][index]['A'] @ answer['solutions_simple_cases'][index] - inputs['simple_cases'][index]['b']).sum())
/tmp/ipykernel_11082/728041831.py:16: RuntimeWarning: invalid value encountered in matmul
  print(np.abs(inputs['pivoting_cases'][index]['A'] @ answer['solution_pivoting_cases'][index] - inputs['pivoting_cases'][index]['b']).sum())


# Task 3: Find inverse of $A$

Use the same approach as with solving SLE, but now instead of $\mathbf b$ for right hand side, insert corresponding Identity matrix $I$ as input into Gaussian Elimination algorithm

Identitiy matrix of size $N \times N$ is built using `numpy.eye(N)` 

In [85]:
def find_inverse(A):
    res = None
    ## YOUR CODE HERE
    # --placeholder start--
    P1, A1, B1 = gauss_elimination(A, np.eye(A.shape[0]), pivoting=True)
    P2, A2, B2 = gauss_elimination(A1[::-1, ::-1], B1[::-1, ::-1], pivoting=False)
    res = B2[::-1, ::-1]
    # --placeholder end--
    return res

You can always check that the matrix that you have found is indeed inverse to $A$ as $A^{-1} A = I$

In [86]:
answer['simple_inverses'] = [
    find_inverse(input['A']) for input in inputs['simple_cases']
]

answer['pivoting_inverses'] = [
    find_inverse(input['A']) for input in inputs['pivoting_cases']
]

# This should be I
print("=== SIMPLE CASES ===")
for index in range(len(inputs['simple_cases'])):
    print(answer['simple_inverses'][0] @ inputs['simple_cases'][0]['A'])

print("=== PIVOTING CASES ===")
for index in range(len(inputs['pivoting_cases'])):
    print(answer['pivoting_inverses'][0] @ inputs['pivoting_cases'][0]['A'])

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 6 and the array at index 1 has size 36

# Task 4: LU decomposition

You can find LU decomposition of the matrix $A$ using Gauss Elimination. Again, it is done using 2 stages of Gauss Elimination:

1. $A \mathbf x = I \mathbf b \rightarrow U \mathbf x = L^* \mathbf b$

2. $I U \mathbf x = L^* \mathbf b \rightarrow L U \mathbf x = I \mathbf b$

But in case you are doing pivoted version, it is a little more tricky. You have to modify somehow the matrix $L$
after the first stage of Gauss Elimination.

In [ ]:
def find_plu(A, pivoting=True):
    A = A.copy().astype(float)
    n = A.shape[0]

    # Step 1: Get P, U, and L_inv using pivoted Gauss elimination
    # The third output is L_inv: inverse of lower triangular matrix L
    P, U, L_inv = gauss_elimination(A, np.eye(n), pivoting=pivoting)

    L_inv = L_inv @ P.T
    # Step 2: Recover L by inverting L_inv via transposition of gauss_elimination result
    # Note: We can use gauss_elimination to "invert" L_inv:
    _, _, L = gauss_elimination(L_inv, np.eye(n), pivoting=False)

    return P.T, L, U

You can always check that your LU decomposition works as

$LU - A = O$ (a zero matrix)

In [ ]:
answer['simple_PLU'] = [
    find_plu(input['A'], pivoting=False) for input in inputs['simple_cases']
]

answer['pivoting_PLU'] = [
    find_plu(input['A'], pivoting=True) for input in inputs['pivoting_cases']
]

answer['nullspace_PLU'] = [
    find_plu(input['A'], pivoting=True) for input in inputs['nullspace_cases']
]


# This should be close to zero
print("=== SIMPLE CASES ===")
for index in range(len(inputs['simple_cases'])):
    print(np.abs(answer['simple_PLU'][index][0] @ answer['simple_PLU'][index][1] @ answer['simple_PLU'][index][2] - inputs['simple_cases'][index]['A']).sum())

print("=== PIVOTING CASES ===")
for index in range(len(inputs['pivoting_cases'])):
    print(np.abs(answer['pivoting_PLU'][index][0] @ answer['pivoting_PLU'][index][1] @ answer['pivoting_PLU'][index][2] - inputs['pivoting_cases'][index]['A']).sum())

print("=== NULLSPACE CASES ===")
for index in range(len(inputs['nullspace_cases'])):
    print(np.abs(answer['nullspace_PLU'][index][0] @ answer['nullspace_PLU'][index][1] @ answer['nullspace_PLU'][index][2] - inputs['nullspace_cases'][index]['A']).sum())

# Finding NullSpace

Now find null-space of the matrix using Gauss Elimination. To do so, analyze the matrix $U$ that appears after Gauss Elimination. Observing this matrix
it is easy to write out the vectors of Null Space.

Note that variable `eps` stands for tolerance. If a value deviates from 0 more than on `eps`, we consider it to be different from `0`.

In [ ]:
def find_null_space(A, pivoting=True, eps=1e-12):
    """
    Return an (n x k) matrix whose columns span Null(A), i.e. { x : A x = 0 }.
    """
    A = A.astype(float)

    ## YOUR CODE HERE ##

    return Z

In [ ]:
answer['nullspace'] = [
    find_null_space(input['A'], pivoting=True) for input in inputs['nullspace_cases']
]

for index in range(len(answer['nullspace'])):
    print(inputs['nullspace_cases'][index]['A'] @ answer['nullspace'][index])

# QR decomposition

Now again we will consider matrix $A^T$ in context of SLE:

$A^T \mathbf x = I \mathbf b$

Out of matrix $A$ we can make an orthonormal matrix $Q$ by orthonoramlizing the rows, this process will turn the $I$ matrix on the right to lower-triangular matrix $L^*$:

$A^T \mathbf x = I \mathbf b \rightarrow Q^T \mathbf x = L^* \mathbf b$

After that we should invert matrix $L^*$ and we will get:

$I Q^T \mathbf x = L^* \mathbf b \rightarrow L Q^T \mathbf x = \mathbf b$

If we transpose that matrix, we will get  QR decomposition of matrix $A$:

$A = Q R,$

where $R$ is upper-triangular matrix.

The task is to:
1. Implement Gramm-Schmidt orthonormalization procedure
2. Combining Gramm-Schmidt algorithm and Gaussian Algorithm, implement QR decomposition

# Task 5: Gramm-Schmidt Algorithm for Matrices

Implement Gramm-Schmidt process for **rows** of matrix $A$ (and do the same transforms to matrix $B$)

In [ ]:
def gramm_schmidt(A, B):
    A = A.copy().astype(float)
    B = B.copy().astype(float)
    ## YOUR CODE HERE
    # --placeholder start--
    for i in range(A.shape[1]):
        for j in range(i):
            factor = np.dot(A[i, :], A[j, :])
            A[i, :] -= factor * A[j, :]
            B[i, :] -= factor * B[j, :]
        factor = np.linalg.norm(A[i, :])
        A[i, :] /= factor
        B[i, :] /= factor
    # --placeholder end--
    return A, B

In [ ]:
answer['gramm_schmidt_simple_cases'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs['simple_cases']
]

answer['gramm_schmidt_pivoting_cases'] = [
    gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs['pivoting_cases']
]

# answer['gramm_schmidt_nullspace_cases'] = [
    # gramm_schmidt(input['A'], np.eye(input['A'].shape[0])) for input in inputs['nullspace_cases']
# ]

print("===== Should be close to 1s: =====")
print((answer['gramm_schmidt_simple_cases'][0][0] ** 2).sum(axis=1))
print()
print("===== Should be close to L: =====")
print(answer['gramm_schmidt_simple_cases'][0][1])

# Task 6: QR decomposition

Implement QR decomposition algorithm. Note that we work on rows on matrix $A$ with our Gauss and Gramm-Schmidt algorithms, but QR decomposition is defined for column matrix 

In [ ]:
def find_qr(A):
    B = np.eye(A.T.shape[0])
    ## YOUR CODE HERE
    # --placeholder start--
    QT, L1 = gramm_schmidt(A.T, B)
    _, I, LT = gauss_elimination(L1, np.eye(A.shape[0]))
    Q = QT.T
    R = LT.T
    # --placeholder end--
    return Q, R

In [ ]:
answer['QR_simple_cases'] = [
    find_qr(input['A']) for input in inputs['simple_cases']
]

print("===== Should be close to O: =====")
print(answer['QR_simple_cases'][0][0] @ answer['QR_simple_cases'][0][1] - inputs['simple_cases'][0]['A'])

In [ ]:
json_tricks.dump(answer, '.answer.json')